In [68]:
import os
import random
import itertools
from tqdm import tqdm
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import scipy.sparse as sp
from scipy import stats, optimize, linalg
from sklearn.preprocessing import normalize
from scipy.linalg import svd
from scipy.sparse import csr_matrix, diags
from scipy.sparse.linalg import svds
from sklearn import metrics, preprocessing, cluster, decomposition
from sklearn_extra.cluster import KMedoids

from node2vec import Node2Vec
import community as community_louvain


def get_numeric_labels(G, communities):
    node_community = {}
    for i, community in enumerate(communities):
        for node in community:
            node_community[node] = i
    return [node_community[node] for node in G.nodes()]


### Methods 
def louvain(G,k):
    return community_louvain.best_partition(G)

def girvan_newman(G, k):
    # Run the Girvan-Newman algorithm
    comp = nx.community.girvan_newman(G)
    # Get the k-th level of the dendrogram
    communities = list(itertools.islice(comp, k))[-1]
    # Create a dictionary to store the clustering
    clustering = {}
    # Assign cluster labels to nodes
    for cluster_id, community in enumerate(communities):
        for node in community:
            clustering[node] = cluster_id
    
    return clustering

def label_propagation(G, k):
    # Note: k is not used in this method, but we keep it for consistency with other methods
    communities = nx.algorithms.community.label_propagation_communities(G)
    clustering = {}
    for i, community in enumerate(communities):
        for node in community:
            clustering[node] = i
    return clustering

# k means 
def create_node_embeddings(G, dimensions=64, walk_length=30, num_walks=200):
    node2vec = Node2Vec(G, dimensions=dimensions, walk_length=walk_length, num_walks=num_walks, workers=4)
    model = node2vec.fit(window=10, min_count=1)
    
    # Create node embeddings
    node_embeddings = {}
    for node in G.nodes():
        node_embeddings[node] = model.wv[node]
    
    return node_embeddings

def kmeans_clustering(G, k):
    # Create node embeddings
    node_embeddings = create_node_embeddings(G)
    
    # Prepare the feature matrix
    X = np.array(list(node_embeddings.values()))
    
    # Normalize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Perform K-means clustering
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X_scaled)
    
    # Create the clustering dictionary
    clustering = {node: label for node, label in zip(node_embeddings.keys(), labels)}
    
    return clustering

def kmedoids_clustering(G, k):
    # Create node embeddings
    node_embeddings = create_node_embeddings(G)
    
    # Prepare the feature matrix
    X = np.array(list(node_embeddings.values()))
    
    # Normalize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Perform K-medoids clustering
    kmedoids = KMedoids(n_clusters=k, random_state=42)
    labels = kmedoids.fit_predict(X_scaled)
    
    # Create the clustering dictionary
    clustering = {node: label for node, label in zip(node_embeddings.keys(), labels)}
    
    return clustering

## Hope 
def BGC(W, args):
    c = np.array(np.sqrt(W.sum(axis=0))).flatten()
    c[c==0] = 1
    c = 1.0/c
    cinv = diags(c)

    F = normalize(W, norm='l1', axis=1)
    B = W.T
    
    Bc = cinv.dot(B)
    
    m, n = W.shape
    dim = min(int(args.dim * args.k), min(m, n) - 1)

    # print(f"dimension={dim}")

    r = np.array(np.sqrt(W.sum(axis=1))).flatten()
    r[r==0] = 1
    r = 1.0/r
    r = diags(r)
    L = Bc.dot(r)
    U, s, V = svds(L, k=dim)
    s = s**2

    alpha = args.alpha
    s = (1.0-alpha)/(1.0-alpha*(s))
    s = np.array(s).flatten()
    s = diags(s).todense()
    U = np.asarray(U.dot(s))  # Convert to numpy array
    U = np.asarray(F.dot(U))  # Convert to numpy array
    U = normalize(U, norm='l2', axis=1)

    # print("start performing k-means...")
    clustering = KMeans(n_clusters=args.k, random_state=1024).fit(U)
    labels = clustering.labels_

    return labels

def FNEM(W, args):
    c = np.array(np.sqrt(W.sum(axis=0))).flatten()
    c[c==0] = 1
    c = 1.0/c
    c = diags(c)

    P = normalize(W, norm='l1', axis=1)
    R = c.dot(W.T)
    r = np.array(np.sqrt(W.sum(axis=1))).flatten()
    r[r==0] = 1
    r = 1.0/r
    r = diags(r)
    R = R.dot(r)

    m, n = W.shape
    dim = min(int(args.dim * args.k), min(m, n) - 1)

    # print(f"dimension={dim}")

    U, s, V = svds(R, k=dim)
    s = s**2

    alpha = args.alpha
    s = (1.0-alpha)/(1.0-alpha*(s))
    s = np.array(s).flatten()
    s = diags(s).todense()
    U = np.asarray(U.dot(s))
    U = np.asarray(P.dot(U))
    U = normalize(U, norm='l2', axis=1)

    U, s, V = svds(U, k=args.k)

    # FNEM rounding
    C = np.zeros((U.shape[0], args.k))
    for i in range(U.shape[0]):
        j = np.argmax(U[i, :])
        C[i, j] = 1
    C = normalize(C, norm='l2', axis=0)

    for _ in range(100):  # You can adjust the number of iterations
        # Update T
        U_T_C = U.T @ C
        Phi, _, Psi = svd(U_T_C)
        T = Phi @ Psi.T

        # Update C
        C = np.zeros((U.shape[0], args.k))
        UT = U @ T
        for i in range(U.shape[0]):
            j = np.argmax(UT[i, :])
            C[i, j] = 1
        C = normalize(C, norm='l2', axis=0)

    labels = np.argmax(C, axis=1)
    return labels


def SNEM(W, args):
    c = np.array(np.sqrt(W.sum(axis=0))).flatten()
    c[c==0] = 1
    c = 1.0/c
    c = diags(c)

    P = normalize(W, norm='l1', axis=1)
    R = c.dot(W.T)
    r = np.array(np.sqrt(W.sum(axis=1))).flatten()
    r[r==0] = 1
    r = 1.0/r
    r = diags(r)
    R = R.dot(r)

    m, n = W.shape
    dim = min(int(args.dim * args.k), min(m, n) - 1)

    # print(f"dimension={dim}")

    U, s, V = svds(R, k=dim)
    s = s**2

    alpha = args.alpha
    s = (1.0-alpha)/(1.0-alpha*(s))
    s = np.array(s).flatten()
    s = diags(s).todense()
    U = np.asarray(U.dot(s))
    U = np.asarray(P.dot(U))
    U = normalize(np.asarray(U), norm='l2', axis=1)

    U, s, V = svds(U, k=args.k)

    # SNEM rounding
    C = np.zeros((U.shape[0], args.k))
    for i in range(U.shape[0]):
        j = np.argmax(U[i, :])
        C[i, j] = 1
    C = normalize(np.asarray(C), norm='l2', axis=0)

    for _ in range(100):  # You can adjust the number of iterations
        # Update T
        T = U.T @ C

        # Update C
        C = np.zeros((U.shape[0], args.k))
        UT = U @ T
        for i in range(U.shape[0]):
            j = np.argmax(UT[i, :])
            C[i, j] = 1
        C = normalize(np.asarray(C), norm='l2', axis=0)

    labels = np.argmax(C, axis=1)
    return labels

def run_clustering(G, algorithm, k, dim=5, alpha=0.2):
    adj_matrix = nx.adjacency_matrix(G)
    # Convert to scipy sparse matrix if it's not already
    W = csr_matrix(adj_matrix)
    
    # Get dimensions
    m, n = W.shape
    # print(f"Matrix shape: {m} x {n}")
    
    # Adjust k if necessary
    k = min(k, min(m, n) - 1)
    
    # Create args object to mimic the original code's structure
    class Args:
        pass
    args = Args()
    args.k = k
    args.dim = dim
    args.alpha = alpha
    
    if algorithm == 'BGC':
        return BGC(W, args)
    elif algorithm == 'FNEM':
        return FNEM(W, args)
    elif algorithm == 'SNEM':
        return SNEM(W, args)
    else:
        raise ValueError("Unknown algorithm. Choose 'BGC', 'FNEM', or 'SNEM'.")

def array_to_dict_clustering(G, array_clustering):
    return {node: int(cluster) for node, cluster in zip(G.nodes(), array_clustering)}

def bgc_clustering(G, k):
    array_result = run_clustering(G, algorithm='BGC', k=k, dim=5, alpha=0.3)
    return array_to_dict_clustering(G, array_result)

def fnem_clustering(G, k):
    array_result = run_clustering(G, algorithm='FNEM', k=k, dim=5, alpha=0.3)
    return array_to_dict_clustering(G, array_result)

def snem_clustering(G, k):
    array_result = run_clustering(G, algorithm='SNEM', k=k, dim=5, alpha=0.3)
    return array_to_dict_clustering(G, array_result)
##
def leading_eigenvector_clustering(G, k):
    """
    Perform leading eigenvector clustering on a graph.
    
    Parameters:
    G (networkx.Graph): The input graph
    k (int): Number of clusters (default is 2)
    
    Returns:
    dict: Mapping of nodes to their cluster assignments
    """
    # Step 1: Compute the Laplacian spectrum
    laplacian_eigenvalues = nx.laplacian_spectrum(G)
    
    # Step 2: Sort eigenvalues and get indices of the k smallest (excluding the smallest)
    sorted_indices = np.argsort(laplacian_eigenvalues)
    k_smallest_indices = sorted_indices[1:k+1]  # Exclude the smallest eigenvalue
    
    # Step 3: Compute the corresponding eigenvectors
    laplacian_matrix = nx.laplacian_matrix(G).todense()
    eigenvectors = np.linalg.eig(laplacian_matrix)[1][:, k_smallest_indices]
    
    # Step 4: Use K-means to cluster the nodes based on their values in the eigenvectors
    kmeans = KMeans(n_clusters=k, n_init=10)
    cluster_labels = kmeans.fit_predict(eigenvectors)
    
    # Step 5: Create a dictionary mapping nodes to their cluster assignments
    return dict(zip(G.nodes(), cluster_labels))
###
def spectral_clustering_graph(G, k):
    # Get the adjacency matrix of the graph as a dense numpy array
    adj_matrix = nx.adjacency_matrix(G).toarray()

    # Convert to float32 to ensure compatibility
    adj_matrix = adj_matrix.astype(np.float32)

    # Create the SpectralClustering object
    spectral_clustering = SpectralClustering(
        n_clusters=k,
        affinity='precomputed',
        n_init=100,
        assign_labels='discretize'
    )

    # Fit the model and get cluster labels
    labels = spectral_clustering.fit_predict(adj_matrix)

    # Create a dictionary to store the results
    clustering_result = {node: int(label) for node, label in zip(G.nodes(), labels)}

    return clustering_result


# Define the evaluation function
def evaluate(true_clustering, pred_clustering):
    # Ensure both dictionaries have the same keys
    assert set(true_clustering.keys()) == set(pred_clustering.keys()), "The dictionaries must have the same keys"
    
    # Convert dictionaries to lists, maintaining order
    nodes = sorted(true_clustering.keys())
    true_labels = [true_clustering[node] for node in nodes]
    pred_labels = [pred_clustering[node] for node in nodes]
    
    # Encode labels as integers
    le = LabelEncoder()
    true_labels_encoded = le.fit_transform(true_labels)
    pred_labels_encoded = le.fit_transform(pred_labels)
    
    # Calculate NMI and ARI (these are invariant to label permutations)
    nmi = normalized_mutual_info_score(true_labels_encoded, pred_labels_encoded)
    ari = adjusted_rand_score(true_labels_encoded, pred_labels_encoded)
    
    # Find the best mapping of predicted labels to true labels
    cm = confusion_matrix(true_labels_encoded, pred_labels_encoded)
    row_ind, col_ind = linear_sum_assignment(-cm)
    best_map = dict(zip(col_ind, row_ind))
    
    # Remap the predicted labels
    pred_labels_remapped = np.array([best_map[label] for label in pred_labels_encoded])
    
    # Calculate accuracy and F1 score with remapped labels
    acc = accuracy_score(true_labels_encoded, pred_labels_remapped)
    f1 = f1_score(true_labels_encoded, pred_labels_remapped, average='weighted')
    
    return {
        "Accuracy": acc,
        "F1 Score": f1,
        "Normalized Mutual Information": nmi,
        "Adjusted Rand Index": ari
    }

def run_clustering_experiment(G, true_clustering, k):
    algorithms = {
        "Louvain": louvain,
        "Girvan_Newman": girvan_newman,
        "Label_Propagation": label_propagation,
        "K-means": kmeans_clustering,
        "K-medoids": kmedoids_clustering,
        "BGC": bgc_clustering,
        "FNEM": fnem_clustering,
        "SNEM": snem_clustering,
        "LE": leading_eigenvector_clustering,
        "SC": spectral_clustering_graph
    }
    
    results = {}
    for name, algorithm in algorithms.items():
        run_results = {'pred_cluster': None, "eval": None, "time": None}
        try:
            # Measure execution time
            start_time = time.time()
            pred_clustering = algorithm(G, k)
            end_time = time.time()
            execution_time = end_time - start_time
            
            # Ensure pred_clustering is a dictionary
            if not isinstance(pred_clustering, dict):
                raise ValueError(f"{name} algorithm did not return a dictionary")
            
            # Store the clustering results and execution time
            run_results['pred_cluster'] = remove_events_from_dict(pred_clustering)
            run_results['time'] = execution_time
            
            # Evaluate the clustering
            evaluation = evaluate(true_clustering, run_results['pred_cluster'])
            run_results['eval'] = evaluation
            results[name] = run_results
            
        except Exception as e:
            print(f"Error in {name}: {str(e)}")
            results[name] = {
                "eval": {
                    "Accuracy": np.nan,
                    "F1 Score": np.nan,
                    "Normalized Mutual Information": np.nan,
                    "Adjusted Rand Index": np.nan
                },
                "time": np.nan
            }
    
    return results

def remove_events_from_dict(input_dict):
    events_to_remove = [f'E{i}' for i in range(1, 15)]
    return {k: v for k, v in input_dict.items() if k not in events_to_remove}  
    
def create_comparison_dataframe(true_clustering, clustering_results):
    # Create a list of dictionaries for each row
    data = []
    
    # Ensure consistent order of names based on true_clustering
    names = list(true_clustering.keys())
    
    for name in names:
        row = {'Name': name, 'True_Clustering': true_clustering[name]}
        
        # Add results from each clustering algorithm
        for algo, results in clustering_results.items():
            if 'pred_cluster' in results and results['pred_cluster'] is not None:
                row[algo] = results['pred_cluster'].get(name, None)
            else:
                row[algo] = None
        
        data.append(row)
    
    # Create the dataframe
    df = pd.DataFrame(data)
    
    # Ensure the correct column order
    columns = ['Name', 'True_Clustering'] + list(clustering_results.keys())
    df = df[columns]
    
    return df
def create_latex_table(df, file_name):
    output_file = "tables/comparison_table"+file_name +".tex"
    # Define the mapping of full names to shortened names
    name_mapping = {
        'True_Clustering':'True',
        'Louvain': 'Louvain',
        'Girvan_Newman': 'GN',
        'Label_Propagation': 'LP',
        'K-means': 'KM',
        'K-medoids': 'KM+',
        'BGC': 'BGC',
        'FNEM': 'FNEM',
        'SNEM': 'SNEM',
        'LE': 'LE',
        'SC': 'SC'
    }
    
    # Rename the columns
    df_renamed = df.rename(columns=name_mapping)
    
    # Reorder columns
    column_order = ['Name', 'True', 'Louvain', 'GN', 'LP', 'KM', 'KM+', 'BGC', 'FNEM', 'SNEM', 'LE', 'SC']
    df_reordered = df_renamed[column_order]
    
    # Generate LaTeX table
    latex_table = df_reordered.to_latex(index=False, escape=False)
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    # Write the LaTeX table to a file
    with open(output_file, 'w') as f:
        f.write(latex_table)
    
    print(f"LaTeX table has been written to {output_file}")
def create_evaluation_comparison_dataframe(results_k2, results_k3):
    # Initialize lists to store data
    algorithms = []
    metrics = ['Accuracy', 'F1 Score', 'Normalized Mutual Information', 'Adjusted Rand Index', 'Time (ms)']
    short_metrics = ['Acc', 'F1', 'NMI', 'ARI', 'Time (ms)']
    metric_mapping = dict(zip(metrics, short_metrics))
    
    data = {metric: {'k=2': [], 'k=3': []} for metric in metrics}

    # Process results for k=2 and k=3
    for algo in results_k2.keys():
        algorithms.append(algo)
        for metric in metrics[:-1]:  # Exclude 'Time (ms)' from this loop
            data[metric]['k=2'].append(results_k2[algo]['eval'][metric])
            data[metric]['k=3'].append(results_k3[algo]['eval'][metric])
        
        # Add time data (convert seconds to milliseconds)
        data['Time (ms)']['k=2'].append(results_k2[algo].get('time', float('nan')) * 1000)
        data['Time (ms)']['k=3'].append(results_k3[algo].get('time', float('nan')) * 1000)

    # Create MultiIndex for columns
    column_index = pd.MultiIndex.from_product([short_metrics, ['k=2', 'k=3']], names=['Metric', 'k'])

    # Create DataFrame
    df = pd.DataFrame(index=algorithms, columns=column_index)

    # Fill DataFrame with data
    for metric, short_metric in metric_mapping.items():
        df[short_metric, 'k=2'] = data[metric]['k=2']
        df[short_metric, 'k=3'] = data[metric]['k=3']

    return df.style.format(decimal='.', thousands=',', precision=2)

def create_latex_comparison_table(styled_df):
    output_file = "tables/comparison_table_k2_k3.tex"
    
    # Convert Styler to LaTeX
    latex_table = styled_df.to_latex(
        hrules=True,
        multicol_align='c',
        clines="all;data",
        sparse_index=False
    )
    
    # Optionally, you can add some LaTeX table formatting
    latex_table = (
        "\\begin{table}[htbp]\n"
        "\\centering\n"
        "\\caption{Comparison of Clustering Algorithms}\n"
        "\\label{tab:comparison}\n"
        "\\small\n"
        + latex_table +
        "\\end{table}"
    )
    
    with open(output_file, 'w') as f:
        f.write(latex_table)
    
    print(f"LaTeX table has been written to {output_file}")

In [57]:
true_clustering = {
     "Evelyn Jefferson":0,
     "Laura Mandeville":0,
     "Theresa Anderson":0,
     "Brenda Rogers":0,
     "Charlotte McDowd":0,
     "Frances Anderson":0,
     "Eleanor Nye":0,
     "Pearl Oglethorpe":0,
     "Ruth DeSand":0,
     "Verne Sanderson":1,
     "Myra Liddel":1,
     "Katherina Rogers":1,
     "Sylvia Avondale":1,
     "Nora Fayette":1,
     "Helen Lloyd":1,
     "Dorothy Murchison":1,
     "Olivia Carleton":1,
     "Flora Price":1
    }


In [58]:
G = nx.davis_southern_women_graph()
a2 = run_clustering_experiment(G, true_clustering, 2)
a3 = run_clustering_experiment(G, true_clustering, 3)

Computing transition probabilities:   0%|          | 0/32 [00:00<?, ?it/s]

Generating walks (CPU: 4): 100%|██████████| 50/50 [00:00<00:00, 193.76it/s]


Computing transition probabilities:   0%|          | 0/32 [00:00<?, ?it/s]

Generating walks (CPU: 4): 100%|██████████| 50/50 [00:00<00:00, 132.90it/s]


Computing transition probabilities:   0%|          | 0/32 [00:00<?, ?it/s]

Generating walks (CPU: 4): 100%|██████████| 50/50 [00:00<00:00, 115.60it/s]


Computing transition probabilities:   0%|          | 0/32 [00:00<?, ?it/s]

Generating walks (CPU: 4): 100%|██████████| 50/50 [00:00<00:00, 125.04it/s]


In [59]:
df_k3 = create_comparison_dataframe(true_clustering, a3)
df_k3

,Name,True_Clustering,Louvain,Girvan_Newman,Label_Propagation,K-means,K-medoids,BGC,FNEM,SNEM,LE,SC
0,Evelyn Jefferson,0,0,0,0,1,1,2,1,1,1,1
1,Laura Mandeville,0,0,0,0,1,1,2,1,1,1,1
2,Theresa Anderson,0,0,0,0,1,1,2,1,1,1,1
3,Brenda Rogers,0,0,0,0,1,1,2,1,1,1,1
4,Charlotte McDowd,0,0,0,0,1,1,2,1,1,1,1
5,Frances Anderson,0,0,0,0,1,1,2,1,1,1,1
6,Eleanor Nye,0,3,0,0,1,0,2,1,1,1,1
7,Pearl Oglethorpe,0,2,0,0,2,0,1,1,1,1,1
8,Ruth DeSand,0,3,0,0,2,0,2,1,1,1,1
9,Verne Sanderson,1,1,1,0,2,0,1,1,1,0,0


In [51]:
df_k2 = create_comparison_dataframe(true_clustering, a)
df_k2

,Name,True_Clustering,Louvain,Girvan_Newman,Label_Propagation,K-means,K-medoids,BGC,FNEM,SNEM,LE,SC
0,Evelyn Jefferson,0,1,0,0,1,0,1,0,0,0,1
1,Laura Mandeville,0,1,0,0,1,1,1,0,0,0,1
2,Theresa Anderson,0,1,0,0,1,0,1,0,0,0,1
3,Brenda Rogers,0,1,0,0,1,1,1,0,0,0,1
4,Charlotte McDowd,0,1,0,0,1,1,1,0,0,0,1
5,Frances Anderson,0,1,0,0,1,0,1,0,0,0,1
6,Eleanor Nye,0,2,0,0,1,0,1,0,0,0,1
7,Pearl Oglethorpe,0,2,0,0,1,0,1,0,0,0,1
8,Ruth DeSand,0,2,0,0,1,0,1,0,0,0,1
9,Verne Sanderson,1,2,0,0,0,0,1,0,0,0,0


In [65]:
# Usage example
df_comparison = create_evaluation_comparison_dataframe(a2, a3)
df_comparison


In [69]:
create_latex_comparison_table(df_comparison)

LaTeX table has been written to tables/comparison_table_k2_k3.tex
